In [6]:
import torch
import numpy as np


x = 1
y = (-x + 1) * 900
y = 900
print(y)

0


In [7]:
import torch
B = 4
labels = (torch.Tensor(list(range(2 * B))) + B) % (2 * B)

In [9]:
labels

tensor([4., 5., 6., 7., 0., 1., 2., 3.])

In [10]:
import torch

def mask_correlated_samples(batch_size, world_size):
    N = 2 * batch_size * world_size
    mask = torch.ones((N, N), dtype=bool)
    mask = mask.fill_diagonal_(0)
    for i in range(batch_size * world_size):
        mask[i, batch_size + i] = 0
        mask[batch_size + i, i] = 0
    return mask

batch_size, world_size = 4, 1
mask = mask_correlated_samples(batch_size, world_size)
print(mask)

tensor([[False,  True,  True,  True, False,  True,  True,  True],
        [ True, False,  True,  True,  True, False,  True,  True],
        [ True,  True, False,  True,  True,  True, False,  True],
        [ True,  True,  True, False,  True,  True,  True, False],
        [False,  True,  True,  True, False,  True,  True,  True],
        [ True, False,  True,  True,  True, False,  True,  True],
        [ True,  True, False,  True,  True,  True, False,  True],
        [ True,  True,  True, False,  True,  True,  True, False]])


In [157]:
import torch
import torch.nn as nn

def mask_correlated_samples(batch_size):
    N = 2 * batch_size
    mask = torch.ones((N, N), dtype=bool)
    mask = mask.fill_diagonal_(0)
    for i in range(batch_size):
        mask[i, batch_size + i] = 0
        mask[batch_size + i, i] = 0
    print(mask)
    return mask


def get_mask(sequence_id):
    b = len(sequence_id)
    mask_id = sequence_id[None, :] - sequence_id[:, None]
    mask_id = mask_id.reshape(-1)
    one_indices = torch.arange(0, b * b, b + 1)
    mask_id[one_indices] = 1
    mask_id = mask_id.reshape(b, -1)
    zero_id = torch.where(mask_id == 0, 0, 1)
    return zero_id


def contrastive_loss(x1, x2, mask):
    assert x1.shape == x2.shape
    B, _ = x1.shape
    x = torch.cat([x1, x2], dim=0)
    logits = nn.CosineSimilarity(dim=2)(x.unsqueeze(1), x.unsqueeze(0))
    
    neg_mask = mask_correlated_samples(B) * mask
    pos_mask = (1 - neg_mask) * (1 - torch.eye(2 * B))

    neg_logits = torch.exp(logits) * neg_mask
    neg_logits = neg_logits.sum(1, keepdim=True)
    exp_logits = torch.exp(logits)
    log_prob = logits - torch.log(exp_logits + neg_logits)
    mean_log_prob_pos = (pos_mask * log_prob).sum(1) / mask.sum(1)
    loss = -mean_log_prob_pos
    loss = loss.mean()

    return loss

x1 = torch.rand(4, 16)
x2 = torch.rand(4, 16)
y = contrastive_loss(x1, x1, torch.cat([get_mask(torch.tensor([1, 2, 1, 4, 5, 3, 4, 5]))], dim=0))
print(y)

tensor([[False,  True,  True,  True, False,  True,  True,  True],
        [ True, False,  True,  True,  True, False,  True,  True],
        [ True,  True, False,  True,  True,  True, False,  True],
        [ True,  True,  True, False,  True,  True,  True, False],
        [False,  True,  True,  True, False,  True,  True,  True],
        [ True, False,  True,  True,  True, False,  True,  True],
        [ True,  True, False,  True,  True,  True, False,  True],
        [ True,  True,  True, False,  True,  True,  True, False]])
tensor(0.4195)


In [160]:
import torch
import torch.nn as nn



def get_mask(sequence_id, n_views):
    b = len(sequence_id)
    mask_id = sequence_id[None, :] - sequence_id[:, None]
    zero_id = torch.where(mask_id == 0, 1, 0)
    one_indices = torch.cat([torch.arange(b // n_views) for i in range(n_views)], dim=0)
    one_indices = (one_indices.unsqueeze(0) == one_indices.unsqueeze(1)).long()
    mask = zero_id + one_indices
    pos_mask = torch.where(mask > 0, 1, 0)
    pos_mask.fill_diagonal_(0)
    neg_mask = torch.where(mask > 0, 0, 1)
    return pos_mask, neg_mask


def contrastive_loss(x, sequence_id, n_views, temperature=1):
    B, _ = x.shape
    logits = nn.CosineSimilarity(dim=2)(x.unsqueeze(1), x.unsqueeze(0))
    pos_mask, neg_mask = get_mask(sequence_id, n_views)
    neg_logits = torch.exp(logits) * neg_mask
    neg_logits = neg_logits.sum(1, keepdim=True)
    exp_logits = torch.exp(logits)
    log_prob = logits - torch.log(exp_logits + neg_logits)
    print(log_prob.shape)
    mean_log_prob_pos = (pos_mask * log_prob).sum(1) / pos_mask.sum(1)
    loss = -mean_log_prob_pos * temperature
    loss = loss.mean()

    return loss

x = torch.rand(9, 16)
n_views = 3
sequence_id = torch.tensor([1, 2, 1, 4, 5, 6, 7, 1, 0])
y = contrastive_loss(x, sequence_id, n_views)
print(y)

torch.Size([9, 9])
tensor(1.8833)


In [ ]:
cd ~/.config/nvim && mkdir patched-fonts
cd patched-fonts
git init
git remote add origin https://github.com/ryanoasis/nerd-fonts
git config core.sparsecheckout true
echo "Hack" >> .git/info/sparse-checkout
git pull --depth 1 origin master 
cd ..
chmod +x install.sh
./install.sh Hack

In [100]:
import numpy as np
base_idx = 450
num_next_frames = 3
frame_skip = 5
num_frames = 900
random_list = [
            *np.arange(0, base_idx - num_next_frames * frame_skip),
            *np.arange(base_idx + num_next_frames * frame_skip + 1, num_frames + 1)
        ]

In [104]:
import torch
a = torch.tensor([1, 2, 3])
b = torch.cat([a for _ in range(3)], dim=0)
b.device

device(type='cpu')

In [117]:
import random

def _sample_indices(num_frame, num_segments, num_frames_per_segment, segment_duration):
    """
    Samples indices for segments.
    """
    indices = []
    time_duration = (num_segments - 1) * segment_duration + num_frames_per_segment
    assert time_duration < num_frame // 2, "Not enough frames to sample"
    assert num_segments % 2 == 1
    center_frame_idx = random.randint(0, num_frame)
    left_frame_idx = center_frame_idx -  segment_duration * num_segments // 2 - num_frames_per_segment // 2
    left_frame_idxs = [left_frame_idx + i * segment_duration for i in range(0, num_segments)]
    for i in left_frame_idxs:
        for j in range(num_frames_per_segment):
            indices.append(i + j)

    return indices


for i in range(10):
    r = _sample_indices(900, 3, 5, 30)
    print(len(r))
    print(r)

15
[494, 495, 496, 497, 498, 524, 525, 526, 527, 528, 554, 555, 556, 557, 558]
15
[148, 149, 150, 151, 152, 178, 179, 180, 181, 182, 208, 209, 210, 211, 212]
15
[357, 358, 359, 360, 361, 387, 388, 389, 390, 391, 417, 418, 419, 420, 421]
15
[381, 382, 383, 384, 385, 411, 412, 413, 414, 415, 441, 442, 443, 444, 445]
15
[353, 354, 355, 356, 357, 383, 384, 385, 386, 387, 413, 414, 415, 416, 417]
15
[449, 450, 451, 452, 453, 479, 480, 481, 482, 483, 509, 510, 511, 512, 513]
15
[797, 798, 799, 800, 801, 827, 828, 829, 830, 831, 857, 858, 859, 860, 861]
15
[806, 807, 808, 809, 810, 836, 837, 838, 839, 840, 866, 867, 868, 869, 870]
15
[375, 376, 377, 378, 379, 405, 406, 407, 408, 409, 435, 436, 437, 438, 439]
15
[269, 270, 271, 272, 273, 299, 300, 301, 302, 303, 329, 330, 331, 332, 333]


In [120]:
import torch
a = torch.arange(0, 12)
a.reshape(3, 4)

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

In [153]:
import numpy as np


def sample(video_list, new_length, num_segments):
    if((len(video_list) - new_length + 1) < num_segments):
        average_duration = (len(video_list) - 5 + 1) // (num_segments)
    else:
        average_duration = (len(video_list) - new_length + 1) // (num_segments)
    offsets = []
    if average_duration > 0:
        offsets += list(np.multiply(list(range(num_segments)), average_duration) + np.random.randint(average_duration,size=num_segments))
    elif len(video_list) > num_segments:
        if((len(video_list) - new_length + 1) >= num_segments):
            offsets += list(np.sort(np.random.randint(len(video_list) - new_length + 1, size=num_segments)))
        else:
            offsets += list(np.sort(np.random.randint(len(video_list) - 5 + 1, size=num_segments)))
    else:
        offsets += list(np.zeros((num_segments,)))
    offsets = np.array(offsets)
    return offsets + 1



video_list = list(range(900))
new_length = 5
num_segments = 3
offsets = sample(video_list, new_length, num_segments)
print(offsets)

[200 526 659]


In [155]:
import torch
M = 4
N = 3

a = torch.ones((M, M)).view(M, M, 1, 1)
b = torch.eye(N).view(1, 1, N, N)
masks = (a * b).transpose(1, 2).reshape(M * N, M * N)
print(masks)

tensor([[1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1.],
        [1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1.],
        [1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1.],
        [1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1.]])


In [5]:
import torch
import torch.nn as nn


ce = nn.CrossEntropyLoss(reduction='none')
x = torch.rand(256, 65536)
y = torch.randint(0, 65536, (256,))
loss = ce(x, y)
print(loss.mean())
print(loss.shape)

tensor(11.1060)
torch.Size([256])


In [54]:
import torch
a = torch.Tensor(1).uniform_(0, 1)
# a=torch.Tensor(2,3).uniform_(-1,1)
torch.tensor([0.0])

tensor([0.])

In [ ]:
def sample_ssl_sequence(sequence: tf.Tensor,
                        num_steps: int,
                        random: bool,
                        stride: int = 1,
                        num_windows: Optional[int] = 2) -> tf.Tensor:
  """Samples two segments of size num_steps randomly from a given sequence.

  Currently it only supports images, and specically designed for video self-
  supervised learning.

  Args:
    sequence: Any tensor where the first dimension is timesteps.
    num_steps: Number of steps (e.g. frames) to take.
    random: A boolean indicating whether to random sample the single window. If
      True, the offset is randomized. Only True is supported.
    stride: Distance to sample between timesteps.
    num_windows: Number of sequence sampled.

  Returns:
    A single Tensor with first dimension num_steps with the sampled segment.
  """
  sequence_length = tf.shape(sequence)[0]
  sequence_length = tf.cast(sequence_length, tf.float32)
  if random:
    max_offset = tf.cond(
        tf.greater(sequence_length, (num_steps - 1) * stride),
        lambda: sequence_length - (num_steps - 1) * stride,
        lambda: sequence_length)

    max_offset = tf.cast(max_offset, dtype=tf.float32)
    def cdf(k, power=1.0):
      """Cumulative distribution function for x^power."""
      p = -tf.math.pow(k, power + 1) / (
          power * tf.math.pow(max_offset, power + 1)) + k * (power + 1) / (
              power * max_offset)
      return p

    u = tf.random.uniform(())
    k_low = tf.constant(0, dtype=tf.float32)
    k_up = max_offset
    k = tf.math.floordiv(max_offset, 2.0)

    c = lambda k_low, k_up, k: tf.greater(tf.math.abs(k_up - k_low), 1.0)
    # pylint:disable=g-long-lambda
    b = lambda k_low, k_up, k: tf.cond(
        tf.greater(cdf(k), u),
        lambda: [k_low, k, tf.math.floordiv(k + k_low, 2.0)],
        lambda: [k, k_up, tf.math.floordiv(k_up + k, 2.0)])

    _, _, k = tf.while_loop(c, b, [k_low, k_up, k])
    delta = tf.cast(k, tf.int32)
    max_offset = tf.cast(max_offset, tf.int32)
    sequence_length = tf.cast(sequence_length, tf.int32)

    choice_1 = tf.cond(
        tf.equal(max_offset, sequence_length),
        lambda: tf.random.uniform((),
                                  maxval=tf.cast(max_offset, dtype=tf.int32),
                                  dtype=tf.int32),
        lambda: tf.random.uniform((),
                                  maxval=tf.cast(max_offset - delta,
                                                 dtype=tf.int32),
                                  dtype=tf.int32))
    choice_2 = tf.cond(
        tf.equal(max_offset, sequence_length),
        lambda: tf.random.uniform((),
                                  maxval=tf.cast(max_offset, dtype=tf.int32),
                                  dtype=tf.int32),
        lambda: choice_1 + delta)
    # pylint:disable=g-long-lambda
    shuffle_choice = tf.random.shuffle((choice_1, choice_2))
    offset_1 = shuffle_choice[0]
    offset_2 = shuffle_choice[1]

  else:
    raise NotImplementedError

  indices_1 = _sample_or_pad_sequence_indices(
      sequence=sequence,
      num_steps=num_steps,
      stride=stride,
      offset=offset_1)

  indices_2 = _sample_or_pad_sequence_indices(
      sequence=sequence,
      num_steps=num_steps,
      stride=stride,
      offset=offset_2)

  indices = tf.concat([indices_1, indices_2], axis=0)
  indices.set_shape((num_windows * num_steps,))
  output = tf.gather(sequence, indices)

  return output

In [65]:
import torch
b = 4
k = 5
pos = torch.randint(0, 3, (b,))
bank = torch.randint(0, 3, (b, k))
print(pos)
print(pos.unsqueeze(1).repeat(1, k))
print(bank)
mask = bank - pos.unsqueeze(1).repeat(1, k)
mask = mask != 0
print(mask)

tensor([1, 2, 1, 2])
tensor([[1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2],
        [1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2]])
tensor([[0, 1, 1, 2, 2],
        [2, 0, 2, 0, 1],
        [0, 2, 2, 1, 2],
        [1, 0, 1, 0, 2]])
tensor([[ True, False, False,  True,  True],
        [False,  True, False,  True,  True],
        [ True,  True,  True, False,  True],
        [ True,  True,  True,  True, False]])


In [8]:
import torch
import torchvision

encoder_q = torchvision.models.resnet50(pretrained=False)
# url = "https://openaipublic.azureedge.net/clip/models/afeb0e10f9e5a86da6080e35cf09123aca3b358a0c3e3b6c78a7b63bc04b6762/RN50.pt"
# pretrained_dict = torch.utils.model_zoo.load_url(url) 
pretrained_dict = torch.jit.load('data/RN50.pt')

In [15]:
r50_dict = {}
for k, v in pretrained_dict.state_dict().items():
    if 'visual' in k:
        r50_dict[k.replace('visual.', '')] = v

In [16]:
encoder_q.load_state_dict(r50_dict)

RuntimeError: Error(s) in loading state_dict for ResNet:
	Missing key(s) in state_dict: "fc.weight", "fc.bias". 
	Unexpected key(s) in state_dict: "conv2.weight", "bn2.weight", "bn2.bias", "bn2.running_mean", "bn2.running_var", "bn2.num_batches_tracked", "conv3.weight", "bn3.weight", "bn3.bias", "bn3.running_mean", "bn3.running_var", "bn3.num_batches_tracked", "attnpool.positional_embedding", "attnpool.k_proj.weight", "attnpool.k_proj.bias", "attnpool.q_proj.weight", "attnpool.q_proj.bias", "attnpool.v_proj.weight", "attnpool.v_proj.bias", "attnpool.c_proj.weight", "attnpool.c_proj.bias". 
	size mismatch for conv1.weight: copying a param with shape torch.Size([32, 3, 3, 3]) from checkpoint, the shape in current model is torch.Size([64, 3, 7, 7]).
	size mismatch for bn1.weight: copying a param with shape torch.Size([32]) from checkpoint, the shape in current model is torch.Size([64]).
	size mismatch for bn1.bias: copying a param with shape torch.Size([32]) from checkpoint, the shape in 

In [3]:
from hachoir.metadata import extractMetadata
from hachoir.parser import createParser

def get_metadata(file_path):
    try :
        metadata = extractMetadata(createParser(file_path))
        return metadata
    except:
        print(file_path)


path = 'data/submission_videos_resized_224/0015d2f4c7c4f8e3b493.avi'
metadata = get_metadata(path)
# 输出文本格式的元数据
print(metadata)
# 输出字典格式的元数据
print(metadata.exportDictionary())
# 输出duration元数据
print(metadata.get('duration'))

Common:
- Duration: 30 sec
- Image width: 224 pixels
- Image height: 224 pixels
- Frame rate: 30.0 fps
- Bit rate: 129.5 Kbit/sec
- Producer: Lavf58.29.100
- Comment: Has audio/video index (14.1 KB)
- MIME type: video/x-msvideo
- Endianness: Little endian
Video stream:
- Duration: 30 sec
- Image width: 224 pixels
- Image height: 224 pixels
- Bits/pixel: 24
- Compression: "FMP4" (fourcc:"FMP4")
- Frame rate: 30.0 fps
{'Common': {'Duration': '30 sec', 'Image width': '224 pixels', 'Image height': '224 pixels', 'Frame rate': '30.0 fps', 'Bit rate': '129.5 Kbit/sec', 'Producer': 'Lavf58.29.100', 'Comment': 'Has audio/video index (14.1 KB)', 'MIME type': 'video/x-msvideo', 'Endianness': 'Little endian'}, 'Video stream': {'Duration': '30 sec', 'Image width': '224 pixels', 'Image height': '224 pixels', 'Bits/pixel': '24', 'Compression': '"FMP4" (fourcc:"FMP4")', 'Frame rate': '30.0 fps'}}
0:00:30


In [2]:
!pip install hachoir

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
     |████████████████████████████████| 647 kB 4.3 MB/s eta 0:00:01


In [4]:
from hachoir.metadata import extractMetadata
from hachoir.parser import createParser

def get_metadata(file_path):
    try :
        metadata = extractMetadata(createParser(file_path))
        return metadata
    except:
        print(file_path)


path = 'data/submission_videos_resized_224/006f0ce7d74832ae977d.avi'
metadata = get_metadata(path)
# 输出文本格式的元数据
print(metadata)
# 输出字典格式的元数据
print(metadata.exportDictionary())
# 输出duration元数据
print(metadata.get('duration'))

Common:
- Duration: 30 sec
- Image width: 224 pixels
- Image height: 224 pixels
- Frame rate: 30.0 fps
- Bit rate: 139.0 Kbit/sec
- Producer: Lavf58.29.100
- Comment: Has audio/video index (14.1 KB)
- MIME type: video/x-msvideo
- Endianness: Little endian
Video stream:
- Duration: 30 sec
- Image width: 224 pixels
- Image height: 224 pixels
- Bits/pixel: 24
- Compression: "FMP4" (fourcc:"FMP4")
- Frame rate: 30.0 fps
{'Common': {'Duration': '30 sec', 'Image width': '224 pixels', 'Image height': '224 pixels', 'Frame rate': '30.0 fps', 'Bit rate': '139.0 Kbit/sec', 'Producer': 'Lavf58.29.100', 'Comment': 'Has audio/video index (14.1 KB)', 'MIME type': 'video/x-msvideo', 'Endianness': 'Little endian'}, 'Video stream': {'Duration': '30 sec', 'Image width': '224 pixels', 'Image height': '224 pixels', 'Bits/pixel': '24', 'Compression': '"FMP4" (fourcc:"FMP4")', 'Frame rate': '30.0 fps'}}
0:00:30


In [4]:
import numpy as np
feat_kp_path = "/home/sunchao/Multi_Agent_Round_2/mouse_r1_1th/results/submissions/submission.npy"
feat_kp = np.load(feat_kp_path, allow_pickle=True)
np.save(feat_kp_path.replace('.npy', '_pickle_false.npy'), feat_kp)

In [5]:
_ = np.load(feat_kp_path.replace('.npy', '_pickle_false.npy'), allow_pickle=False)

ValueError: Object arrays cannot be loaded when allow_pickle=False

In [6]:
import numpy as np
feat_kp_path = "/home/sunchao/Multi_Agent_Round_2/mouse_r1_1th/results/submissions/submission.npy"
feat_kp = np.load(feat_kp_path, allow_pickle=False)

In [7]:
feat_kp.shape

(3294000, 59)

In [ ]:
import numpy as np

In [8]:
def f(a, b, c):
    return a + b + c

f(1, 2, 3, 4)


TypeError: f() takes 3 positional arguments but 4 were given